In [ ]:
!pip install flask
!pip install flask-socketio
!pip install "python-socketio[asyncio_client]"

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

from flask import Flask, render_template
from flask_socketio import SocketIO, send, emit

import time

In [ ]:
def partition(initial_model: tf.keras.Model, part: int) -> None:
    part1 = initial_model.layers[:part]
    part2 = initial_model.layers[part:]
    
    part1_model = keras.Model(
        inputs=model.input,
        outputs=[layer.output for layer in part1]
    )

    part2_model = keras.Model(
        inputs=keras.Input(tensor=part1_model.output),
        outputs=[layer.output for layer in part2]
    )

    part1_model.save("models/part1")
    part2_model.save("models/part2")



In [ ]:
in_list = [[1, 2], [3, 4], [5, 6]]
inpt = tf.convert_to_tensor(in_list)

layer_a = tf.keras.layers.Dense(3, kernel_initializer=tf.constant_initializer(1.))
layer_b = tf.keras.layers.Dense(1, kernel_initializer=tf.constant_initializer(1.))

In [ ]:
model = tf.keras.Sequential()
model.add(layer_a)
model.add(layer_b)
out = model(inpt)
print("Local run: {}".format(out))
partition(model, 1)

In [ ]:
app = Flask(__name__)
app.config['SECRET_KEY'] = 'secret!'
socketio = SocketIO(app)

In [ ]:
@socketio.on('from node', namespace="/net")
def handle_data(node, data):
    print("Data handled")
    if node == "part1":
        print("Sending to part2")
        socketio.emit('to part2', {"server": data}, namespace="/net")
    elif node == "part2":
        print("Done distributing, result is {}".format(data))
        exit(0)

@socketio.on('connect', namespace="/net")
def client_connect(auth):
    print("Auth: ", auth)
    print("{} has connected".format(auth["name"]))
    if auth["name"] == "part2":
        print("Emitting to part1")
        socketio.emit('to part1', {"server": in_list}, namespace="/net")

In [ ]:
if __name__ == '__main__':
    socketio.run(app)